# 📋 Documentación Completa del Script de Limpieza de Datos
## Análisis RCP Transtelefónica

Este notebook documenta paso a paso todo lo que hace el script `cleandata.py` para procesar los datos del estudio de RCP (Reanimación Cardiopulmonar) Transtelefónica.

### 🎯 Objetivo del Script
El script toma los datos raw del estudio y los transforma en un dataset limpio y estructurado, listo para el análisis estadístico científico. El objetivo principal es evaluar si la RCP guiada por teléfono mejora:
- **ROSC** (Return of Spontaneous Circulation - Retorno de Circulación Espontánea)
- **Supervivencia a 7 días**

### 📁 Archivos Involucrados
- **Entrada**: `data/rawdata_2year.csv` (datos raw de 2 años)
- **Salida**: `data/cleaned_data.csv` (datos limpios y procesados)

---

## 🔄 Pipeline de Procesamiento: 10 Pasos

El script ejecuta **10 pasos secuenciales** para transformar los datos raw en datos listos para análisis:

```
Datos Raw → [10 Pasos] → Datos Limpios
```

### Flujo General:
1. 📖 **Lectura y renombrado** de columnas
2. 🔢 **Cálculo de tiempos** (llegada de unidad, duración RCP)
3. 💓 **Categorización de ritmos** (desfibrilable vs no desfibrilable)
4. 👨‍⚕️ **Clasificación de respondientes** (sanitario, lego, policía, bombero)
5. 🚫 **Filtrado de casos traumáticos**
6. 🔗 **Fusión de datos SVB** en registros SVA
7. 📞 **Filtrado RCP transtelefónica** (casos válidos)
8. 🧠 **Cálculo de CPC** (Cerebral Performance Category)
9. 💗 **Determinación de supervivencia** a 7 días
10. 📊 **Generación de resumen** estadístico

## 📖 Paso 1: Lectura y Renombrado de Columnas

### ¿Qué hace?
- Lee el archivo `rawdata_2year.csv`
- Renombra las columnas a nombres más claros y estándar

### Mapeo de Columnas:
```python
column_mapping = {
    'fecha': 'FECHA',
    'sexo': 'SEXO', 
    'edad': 'EDAD',
    'quien_hace_rcp': 'QUIEN_HACE_RCP',
    'rcp_transtelef': 'RCP_TRANSTELEFONICA',
    'tiempo_llegada_unidad': 'TIEMPO_LLEGADA_UNIDAD',
    'desa_externo': 'DESA_EXTERNO',
    'ritmo_inicial': 'RITMO_INICIAL',
    'codigo_patologico': 'CODIGO_PATOLOGICO',
    'hora_inicio_rcp': 'HORA_INICIO_RCP',
    'hora_fin_rcp': 'HORA_FIN_RCP'
}
```

### ¿Por qué es importante?
- Estandariza los nombres para facilitar el análisis
- Convierte nombres descriptivos en español
- Facilita la legibilidad del código

## ⏱️ Paso 2: Cálculo de Tiempos

### ¿Qué hace?
Calcula dos tiempos críticos para el análisis:

#### A) Tiempo de Llegada de la Unidad (segundos)
- Convierte minutos a segundos
- Limpia valores no válidos

#### B) Tiempo de Duración de RCP (segundos)
- Calcula diferencia entre `HORA_FIN_RCP` y `HORA_INICIO_RCP`
- Maneja diferentes formatos de tiempo:
  - HH:MM:SS (ej: "14:30:25")
  - HH:MM (ej: "14:30")
  - Solo números (minutos)

### Ejemplo de Cálculo:
```
HORA_INICIO_RCP: "14:30:00"
HORA_FIN_RCP: "14:45:30"
TIEMPO_RCP_SEGUNDOS: 930 segundos (15 min 30 seg)
```

### ¿Por qué es importante?
- El tiempo es un factor pronóstico crucial en paradas cardiacas
- Permite análisis de supervivencia basado en tiempos
- Estandariza unidades (todo en segundos)

## 💓 Paso 3: Categorización de Ritmos

### ¿Qué hace?
Clasifica el ritmo inicial en dos categorías clínicamente relevantes:

#### Ritmos Desfibrilables (mejor pronóstico):
- **FV**: Fibrilación Ventricular
- **TV**: Taquicardia Ventricular
- **TVsp**: Taquicardia Ventricular sin pulso

#### Ritmos No Desfibrilables (peor pronóstico):
- **Asistolia**
- **AESP**: Actividad Eléctrica Sin Pulso
- **Bradicardia**
- Otros ritmos

### Código de Clasificación:
```python
def categorize_rhythm(rhythm):
    if pd.isna(rhythm):
        return False
    rhythm_lower = str(rhythm).lower()
    return rhythm_lower in ['fv', 'tv', 'tvsp']
```

### ¿Por qué es importante?
- El ritmo inicial es el predictor más fuerte de supervivencia
- Ritmos desfibrilables tienen ~80% de ROSC vs ~30% no desfibrilables
- Permite estratificación en el análisis estadístico

## 👨‍⚕️ Paso 4: Clasificación de Respondientes (MEJORADA)

### ¿Qué hace?
Clasifica quién realizó la RCP usando **códigos de emergencia** y **terminología ampliada**:

#### Sistema de Clasificación Prioritaria:
1. **SANITARIO** (prioridad más alta)
2. **BOMBERO** 
3. **POLICÍA**
4. **LEGO** (ciudadano común)

### Códigos de Emergencia Específicos:

#### 🚓 POLICÍA:
- **091**: Policía Nacional
- **092**: Policía Local
- **062**: Guardia Civil
- Términos: "policia", "guardia civil", "agente"

#### 🚒 BOMBEROS:
- **080**: Bomberos
- Términos: "bombero", "beta" (bomberos voluntarios)

#### 🚑 SANITARIOS:
- **061**: SAMUR (Madrid)
- **060**: Emergencias sanitarias regionales
- **SUMMA**: Servicio de Urgencias Madrid
- **SAMUR**: Servicio de Asistencia Municipal de Urgencia
- **SVB**: Soporte Vital Básico
- Términos: "medico", "enfermero", "sanitario", "hospital"

#### 👥 LEGOS (Terminología Ampliada):
- "lego", "ciudadano", "testigo", "persona"
- "alertante", "demandante", "llamante"
- Familiares, transeúntes, etc.

### ¿Por qué es importante?
- **Precisión mejorada**: De 10.1% a 50.8% sanitarios con códigos
- **Análisis estratificado**: Calidad de RCP varía por tipo de respondiente
- **Validez científica**: Clasificación más precisa para conclusions válidas

## 🚫 Paso 5: Filtrado de Casos Traumáticos

### ¿Qué hace?
Excluye casos de parada cardiaca de origen **traumático** del análisis.

#### Criterios de Exclusión:
Busca estas palabras clave en cualquier columna de texto:
- "trauma", "traumatic", "traumático"
- "accidente", "atropello"
- "precipitación", "caída"
- "herida", "lesión"
- "politraumatismo"
- "ahogamiento", "asfixia"

### ¿Por qué es importante?
- **Homogeneidad**: El estudio se centra en paradas cardiacas **médicas**
- **Validez**: Las paradas traumáticas tienen fisiopatología diferente
- **Estándares**: Siguiendo guidelines internacionales (AHA, ERC)
- **Poder estadístico**: Evita confusión por diferentes etiologías

## 🔗 Paso 6: Fusión de Datos SVB en SVA

### ¿Qué hace?
Cuando hay **dos registros** para la misma fecha (SVB y SVA), los **fusiona** en uno solo.

#### Lógica de Fusión:
```python
# Para cada fecha con múltiples registros:
# 1. Identificar registro SVB (CODIGO_PATOLOGICO contiene 'SVB')
# 2. Identificar registro SVA (sin 'SVB')
# 3. Mantener datos del SVA
# 4. Sobrescribir con datos del SVB cuando sean más completos
```

#### Ejemplo:
```
Fecha: 2023-01-15
Registro SVB: RCP_TRANSTELEFONICA=True, ROSC=No documentado
Registro SVA: RCP_TRANSTELEFONICA=No documentado, ROSC=True

Resultado: RCP_TRANSTELEFONICA=True, ROSC=True
```

### ¿Por qué es importante?
- **Evita duplicados**: Un caso = un registro
- **Maximiza información**: Combina datos de ambos equipos
- **Precisión**: SVB tiene mejor información de RCP transtelefónica
- **Validez**: Análisis sin sesgos por registros duplicados

## 📞 Paso 7: Filtrado RCP Transtelefónica

### ¿Qué hace?
Aplica **control de calidad** para casos de RCP transtelefónica.

#### Criterios de Validación:
```python
def filter_rcp_transtelefonica(df):
    # Solo mantener casos donde:
    # 1. RCP_TRANSTELEFONICA está claramente documentada (True/False)
    # 2. No hay valores ambiguos o missing en esta variable
    return df[df['RCP_TRANSTELEFONICA'].notna()]
```

### ¿Por qué es importante?
- **Variable principal**: RCP transtelefónica es el endpoint primario
- **Calidad de datos**: Solo casos con documentación clara
- **Poder estadístico**: Evita bias por missing data
- **Reproducibilidad**: Criterios claros de inclusión

## 🧠 Paso 8: Cálculo de CPC (Cerebral Performance Category)

### ¿Qué hace?
Extrae y clasifica el **estado neurológico** del paciente usando la escala CPC.

#### Escala CPC:
- **CPC 1**: Buen rendimiento neurológico (vida normal)
- **CPC 2**: Discapacidad moderada (independiente para AVD)
- **CPC 3**: Discapacidad severa (dependiente)
- **CPC 4**: Estado vegetativo
- **CPC 5**: Muerte cerebral o muerte

#### Lógica de Extracción:
```python
def extract_cpc(codigo_patologico):
    # Busca patrones como:
    # "C.1.0", "CPC 1", "CPC1", etc.
    pattern = r'(?:C\.|CPC\s*)(\d)'
    match = re.search(pattern, str(codigo_patologico))
    if match:
        return int(match.group(1))
    return None
```

### ¿Por qué es importante?
- **Outcome neurológico**: No solo supervivencia, sino calidad de vida
- **Endpoint secundario**: CPC 1-2 = supervivencia neurológicamente íntegra
- **Relevancia clínica**: Lo que realmente importa a pacientes/familias

## 💗 Paso 9: Determinación de Supervivencia a 7 Días

### ¿Qué hace?
Determina si el paciente **sobrevivió** los primeros 7 días tras la parada cardiaca.

#### Lógica de Clasificación:
```python
def determine_survival(codigo_patologico):
    if pd.isna(codigo_patologico):
        return False
    
    codigo_str = str(codigo_patologico).upper()
    
    # Superviviente si:
    survival_patterns = ['C.1.', 'C.2.', 'C.3.', 'CPC 1', 'CPC 2', 'CPC 3']
    
    # No superviviente si:
    death_patterns = ['C.4.', 'C.5.', 'CPC 4', 'CPC 5', 'EXITUS', 'FALLEC']
```

#### Ejemplos:
- `"C.1.0"` → **Superviviente** (CPC 1)
- `"C.5.0"` → **No superviviente** (CPC 5 = muerte)
- `"EXITUS"` → **No superviviente**
- `"C.3.1"` → **Superviviente** (CPC 3, con discapacidad pero vivo)

### ¿Por qué es importante?
- **Endpoint primario**: Supervivencia es el outcome más importante
- **Estándar internacional**: 7 días es el período estándar de seguimiento
- **Simplicidad**: Variable binaria clara para análisis estadístico

## 📊 Paso 10: Generación de Resumen Estadístico

### ¿Qué hace?
Genera un **reporte completo** con todas las estadísticas descriptivas del dataset limpio.

#### Secciones del Resumen:

1. **📊 Total de casos** analizados
2. **👥 Demografía**: edad (media ± SD, rango), sexo (n, %)
3. **📞 RCP Transtelefónica**: con/sin (n, %)
4. **👨‍👩‍👧‍👦 RCP de Testigos**: con/sin (n, %)
5. **🚑 Tipo de Respondiente**: sanitario/lego/policía/bombero (n, %)
6. **⚡ DESA Externo**: con/sin (n, %)
7. **💓 Ritmo Inicial**: desfibrilable/no desfibrilable (n, %)
8. **⏱️ Tiempos**: llegada unidad, duración RCP (media ± SD, mediana)
9. **💗 ROSC**: con/sin (n, %)
10. **🏥 Supervivencia a 7 días**: sí/no (n, %)
11. **🧠 CPC**: distribución por categorías (n, %)
12. **🔍 Análisis cruzado**: ROSC y supervivencia por subgrupos

### Ejemplo de Output:
```
📊 TOTAL DE CASOS ANALIZADOS: 622

👥 DEMOGRAFÍA:
   • Edad promedio: 64.6 ± 17.0 años
   • Sexo: Masculino 463 (74.4%), Femenino 136 (21.9%)

🔍 ANÁLISIS CRUZADO CLAVE:
   • ROSC con RCP transtelefónica: 33/62 (53.2%)
   • ROSC sin RCP transtelefónica: 203/560 (36.2%)
```

### ¿Por qué es importante?
- **Validación**: Confirma que los datos son consistentes
- **Tabla 1**: Base para la caracterización de la muestra en el paper
- **Hallazgos preliminares**: Primeras evidencias de efectividad
- **Control de calidad**: Detecta problemas en el procesamiento

## 🎯 Resultados del Procesamiento

### Dataset Final (`cleaned_data.csv`):
- **622 casos** de parada cardiaca extrahospitalaria no traumática
- **13 variables** listas para análisis estadístico
- **Datos de calidad** con criterios de inclusión claros

### Variables Principales:
```python
[
    'FECHA',                    # Fecha del evento
    'EDAD',                     # Edad del paciente (años)
    'SEXO',                     # Sexo (M/F)
    'RCP_TRANSTELEFONICA',      # RCP guiada por teléfono (True/False)
    'RCP_TESTIGOS',             # RCP por testigos (True/False)
    'TIPO_RESPONDIENTE',        # Quien hizo RCP (sanitario/lego/policia/bombero)
    'TIEMPO_LLEGADA_UNIDAD',    # Tiempo hasta llegada SEM (segundos)
    'TIEMPO_RCP_SEGUNDOS',      # Duración total RCP (segundos)
    'DESA_EXTERNO',             # Uso de DESA (True/False)
    'RITMO_DESFIBRILABLE',      # Ritmo inicial desfibrilable (True/False)
    'ROSC',                     # Retorno circulación espontánea (True/False)
    'SOBREVIVE_7DIAS',          # Supervivencia 7 días (True/False)
    'CPC'                       # Estado neurológico (1-5)
]
```

### Hallazgos Clave:
- **RCP Transtelefónica**: 10% de casos, con 53.2% ROSC vs 36.2% sin ella
- **Supervivencia**: 32.3% vs 18.9% a favor de RCP transtelefónica
- **Ritmo desfibrilable**: Solo 9% pero con 82.1% de ROSC
- **Respondientes**: 55.3% sanitarios, 35.2% legos (clasificación mejorada)

## 🔧 Aspectos Técnicos Importantes

### Librerías Utilizadas:
```python
import pandas as pd      # Manipulación de datos
import re               # Expresiones regulares
from datetime import datetime  # Manejo de tiempos
```

### Manejo de Missing Data:
- **Estrategia conservadora**: Mantener NaN cuando no hay información clara
- **Imputación selectiva**: Solo para variables críticas con lógica clínica
- **Documentación**: Reportar % de missing para cada variable

### Control de Calidad:
- **Validación de rangos**: Edad 0-120, tiempos > 0
- **Consistencia lógica**: ROSC vs supervivencia, CPC vs outcome
- **Detección de outliers**: Tiempos extremos, casos atípicos

### Reproducibilidad:
- **Código documentado**: Cada función con docstring
- **Versionado**: Control de cambios en el procesamiento
- **Trazabilidad**: Log de transformaciones aplicadas

## 📈 Próximos Pasos: Análisis Estadístico

Con los datos limpios, el siguiente paso es el **análisis estadístico completo**:

### 1. Análisis Descriptivo:
- Caracterización de la muestra (Tabla 1)
- Distribuciones de variables continuas
- Frecuencias de variables categóricas

### 2. Análisis Bivariado:
- **Chi-cuadrado**: RCP transtelefónica vs ROSC/supervivencia
- **t-test**: Edad, tiempos según outcomes
- **Tablas de contingencia**: 2x2 para OR crudo

### 3. Análisis Multivariado:
- **Regresión logística**: Ajustada por edad, sexo, tiempo llegada
- **Forest plots**: ORs con intervalos de confianza
- **Curvas ROC**: Capacidad predictiva del modelo

### 4. Visualizaciones:
- Gráficos de barras comparativos
- Boxplots de tiempos por grupo
- Curvas de supervivencia (Kaplan-Meier)

### 5. Sensibilidad:
- Análisis por subgrupos (edad, ritmo inicial)
- Validación cruzada de modelos
- Análisis de missing data

---

## ✅ Conclusión

El script `cleandata.py` ha transformado exitosamente los datos raw en un dataset científicamente válido, listo para análisis estadístico riguroso. La **clasificación mejorada de respondientes** y el **control de calidad** aseguran la validez de las conclusiones que se obtengan.

**Dataset final**: 622 casos, 13 variables, datos de alta calidad para evaluar la efectividad de la RCP transtelefónica.